In [2]:
# First step, import libraries.
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt


In [3]:

df = pd.read_csv('C:/Users/User/Desktop/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
group = df.groupby('date')
Real_Price = group['Weighted_Price'].mean()


In [4]:
#Real_Price

date
2011-12-31        4.472579
2012-01-01        4.680778
2012-01-02        5.000000
2012-01-03        5.145917
2012-01-04        5.195443
2012-01-05        6.124426
2012-01-06        6.540973
2012-01-07        6.192222
2012-01-08        6.842292
2012-01-09        6.697618
2012-01-10        6.558201
2012-01-11        7.006282
2012-01-12        6.877167
2012-01-13        6.760737
2012-01-14        6.427134
2012-01-15        6.781076
2012-01-16        7.061111
2012-01-17        6.910459
2012-01-18        6.538131
2012-01-19        6.323160
2012-01-20        6.655063
2012-01-21        6.887750
2012-01-22        6.614472
2012-01-23        6.543410
2012-01-24        6.506604
2012-01-25        6.511689
2012-01-26        6.258669
2012-01-27        5.808681
2012-01-28        5.074361
2012-01-29        5.650951
                  ...     
2017-12-10    14132.601711
2017-12-11    16200.092355
2017-12-12    16729.848282
2017-12-13    16539.760724
2017-12-14    16340.216884
2017-12-15    17364.190

In [4]:
# split data
prediction_days = 30
df_train= Real_Price[:len(Real_Price)-prediction_days]
df_test= Real_Price[len(Real_Price)-prediction_days:]


In [5]:
# Data preprocess
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set) # 0 mean and 1 standard error to center the data
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))


In [6]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Initialising the RNN
regressor = Sequential() # seq model which stacks all the layer

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None, 1))) # no of units used for lstm cell

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
#adam : extension of stochastic gradient decent ; bias correction
# lerning rate is sigular though the weigths gets updated ; per parametr learning rate with second moment
# adaptive grad;rms prop
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error') # loss function we need to minimize

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100) # 5 examples for each iterations and no of iterations 100


C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


ModuleNotFoundError: No module named 'tensorflow'

In [10]:
# Making the predictions and reshape the data, tranform the data
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)


In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(test_set, color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('BTC Price Prediction', fontsize=40)
df_test = df_test.reset_index()
x=df_test.index
labels = df_test['date']
plt.xticks(x, labels, rotation = 'vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()
